# Market marker - price and fill orders

Inspo:
- https://github.com/cornwarecjp/marketmaker/blob/master/marketmaker.py
- https://money.stackexchange.com/questions/24442/how-are-futures-contracts-setup-at-an-exchange
- https://pdfs.semanticscholar.org/e0f7/35e27f5f89ecc6dd7d7cad8f6410aff6270f.pdf
- https://www.researchgate.net/profile/Bastien-Baldacci/publication/334760288_Algorithmic_market_making_the_case_of_equity_derivatives/links/5e1cf0974585159aa4ce78b8/Algorithmic-market-making-the-case-of-equity-derivatives.pdf

3 possibilities for liquidity:
1. Treat the underlying asset (the music time series) as the actual asset value $S_t$, which is not tradeable - like weather data. Issue contracts on this underlying non-tradeable asset and build the market around that.
2. Issue shares in predictions for the underlying asset according to a stochastic process, auction off to agents, and also play the role of market-maker
    - At some point, could jointly optimize the stochastic process, the auction structure and market-making mechanism (simplest: Hanson's LMSR).
    - Prep for blockchain style model.
3. Fully predictive approach: LMSR where you buy shares for predicting outcome A vs. outcome B. At the end of the outcome, financial reward/loss.
    - Number of outstanding shares (corresponding to discrete predicted outcomes) starts at 0 and only grows.
        - https://mason.gmu.edu/~rhanson/mktscore.pdf
        - http://blog.oddhead.com/2006/10/30/implementing-hansons-market-maker/ - for selling too
    - Could build derivatives on top of this later, e.g. make shares tradeable + right to buy/sell share later for given price.
    
Focus on #3 for now - the most feasible.
1. Start with simplest decision problem: predict UP vs. DOWN (buying N shares of up vs. down) for K periods from now.

In [2]:
from __future__ import division
from more_itertools import peekable
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as scp
import pickle
import magenta
import os, time, re
%matplotlib inline
from IPython.core.display import display, HTML
### change width of notebook display
display(HTML("<style>.container { width:70% !important; }</style>"))

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# from IPython.core.magic import register_cell_magic
# @register_cell_magic('handle')
# def handle(line, cell):
#     try:
#         exec(cell)
#     except Exception as e:
#         raise # if you want the full trace-back in the notebook

JUPYTER_PICKLE_FILE = "config/shared_jupyter_data.pkl"
def write_shared_jupyter(key, value, path=JUPYTER_PICKLE_FILE, overwrite=False):
    if (os.path.exists(path)):
        with open(path, "rb") as fp:
            shared_jupyter_data = pickle.load(fp)
        if overwrite:
            shared_jupyter_data = {key: value}
        else:
            shared_jupyter_data[key] = value
    else:
        shared_jupyter_data = {key: value}
    with open(path, 'wb') as fp: 
        pickle.dump(shared_jupyter_data, fp)

def read_shared_jupyter(key=None, path=JUPYTER_PICKLE_FILE):
    if (os.path.exists(path)):
        with open(path, "rb") as fp:
            shared_jupyter_data = pickle.load(fp)
            if key is not None:
                if key in shared_jupyter_data:
                    return(shared_jupyter_data[key])
                else:
                    print("Not found!")
                    return(None)
            else:
                return(shared_jupyter_data)
    else:
        print("No data")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

FIG_WIDTH = 1200
FIG_HEIGHT = 800

PITCH_MIN = 20
PITCH_MAX = 120
VELOCITY_MIN = 0
VELOCITY_MAX = 120

def hheader(x):
    print("#########################################")
    print("### {}".format(x))
    print("#########################################")

# Magenta dependencies:
# https://github.com/magenta/magenta

# Magenta uses pretty_midi to deal with midi files
import pretty_midi

C:\Users\echow\Miniconda3\envs\tensorflow\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\echow\Miniconda3\envs\tensorflow\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\echow\Miniconda3\envs\tensorflow\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Could

# Simulate orders for market-making

Orders will be to buy or sell shares of A, B, C ... which are the predicted discrete outcomes.

Start simple by betting whether will go up or down for (horizon) periods from now.

Specifically, a bet is a tuple (quantity, horizon, outcome).

https://lobsterdata.com/info/DataStructure.php

In [114]:

### order book template data
order_book_msgs = pd.read_csv("order_book/AMZN_2012-06-21_34200000_57600000_message_1.csv", header=None)
order_book_msgs.columns = ['ts_secs_after_midnight', "event_type", "order_id", "bet_quantity", "price", "direction"]
order_book_msgs['price'] = order_book_msgs['price'] / 1000.0 # in dollars

### only look at submitted orders for now
order_book_msgs = order_book_msgs.loc[order_book_msgs['event_type'] == 1,].reset_index(drop=True)
order_book_msgs['bet_outcome'] = order_book_msgs['direction'] # +1 = up, -1 = down

### generate random order times (monotonically increasing) - something plausible
# order_book_msgs = order_book_msgs.reset_index()
# order_book_msgs['order_book_streaming_ts'] = order_book_msgs['index']
order_book_msgs['order_book_streaming_ts'] = np.sort(np.random.gamma(1, np.sqrt(order_book_msgs.shape[0]), order_book_msgs.shape[0]))
# start at time 1000 so can weave together streaming order book data and also streaming music data
order_book_msgs['order_book_streaming_ts'] = order_book_msgs['order_book_streaming_ts'] + 1000

### simulate bet horizon
order_book_msgs['bet_horizon'] = order_book_msgs['order_book_streaming_ts'] + np.random.randint(20, 30, order_book_msgs.shape[0])

order_book_msgs = order_book_msgs.drop(columns=['ts_secs_after_midnight', 'direction', "price"])

# price here is set by the market-maker entirely (agents are price-takers)
# maybe later can incorporate price-power on the part of the agents

### simulate some agent IDs
### (assume agents have enough wealth)
SIM_N_AGENTS = 100
order_book_msgs['agent_id'] = np.random.randint(0, SIM_N_AGENTS, order_book_msgs.shape[0])
order_book_msgs = order_book_msgs[['order_book_streaming_ts', 'agent_id', 'order_id', 'bet_outcome', 'bet_horizon', 'bet_quantity']]

order_book_msgs.to_csv("order_book/simulated_order_book.csv")

order_book_evolv = pd.read_csv("order_book/AMZN_2012-06-21_34200000_57600000_message_1.csv")

KeyError: 'order_book_streaming_ts'

In [112]:
order_book_msgs.head()

,order_book_streaming_ts,agent_id,order_id,bet_outcome,quantity
0,0.009704,96,11885113,1,21
1,0.016476,72,16207239,-1,100
2,0.020087,42,16208720,-1,50
3,0.027996,49,16240373,-1,100
4,0.032167,57,16240856,-1,100


# Automated market-making: read, price and fill orders

Remember, in this simple model, we just issue new shares.

For the real line:
http://yiling.seas.harvard.edu/wp-content/uploads/gao-wine09-interval.pdf

Could use Monte Carlo to approximate those integrals.

In [113]:
""" Set up order book stream """

### for reading in chunks
from collections import deque

def csvStream(csvfile):
    csv_stream = pd.read_csv(csvfile, index_col=0, iterator=True)
    return(csv_stream)

### Always fill 1 order at a time
def nextChunk(csvStream, chunksize=1):
    return(csvStream.get_chunk(chunksize))

### Create order book stream
orderBookStream = csvStream("order_book/simulated_order_book.csv")

In [106]:
""" Automated market-making: set up parameters for market-maker
"""

MARKET_MAKER_WEALTH = 10**6

In [107]:
""" Automated market-making: LMSR
"""

### LMSR parameters
b = 1000
quantity_shares_up = 0
quantity_shares_down = 0
def execute_order(quantity, bet_outcome="up"):
    if bet_outcome == "up":
        cost_before = 

def quote_curr_price():
    return()


print("Upper bound on market-maker loss: {}".format(b * np.log(2)))


iterations = 0
while True:
    ### Read in current order
    currOrder = nextChunk(orderBookStream)
    if currOrder is None:
        print(">> End of stream!")
        break
    
    if (iterations % 1000 == 0):
        print("Iterations {} ...".format(iterations+1))

    """ Analysis with current order here """

    ### Calculate price
    
    ### Write out price predictio to shared jupyter space
        
    ### store via Jupyter so other notebooks can access
    ### (later work might actually build an API)
    raise Exception()
    write_shared_jupyter("currOrder", currOrder, overwrite=False)
    
    ### read trading decisions from other agents

    ### Take a short break between analyses (so plotly can catch up)
    ### should be >> plot auto-update interval so that all plots
    ### update basically at the same time. 
    time.sleep(1) # 1 second is comfortable for nice UI
    iterations += 1

SyntaxError: invalid syntax (<ipython-input-107-34c9deb01616>, line 8)

In [84]:
currOrder

,order_book_streaming_ts,agent_id,order_id,direction,size,price
605,3.782578,89,27870490,sell,100,2238.7
